_This project was developed independently as part of Code Institute’s Predictive Analytics Project. Any datasets or templates used are openly provided by the course or via public sources like Kaggle. All commentary and code logic are my own._

# 05 Model Training And Evaluation

## Objective

This notebook aims to train, evaluate, and compare multiple machine learning models to predict house sale prices using features engineered in earlier stages.
Given our decision to retain outliers, tree-based models (especially ensemble methods) are expected to perform better.

### Inputs  
- `/data/processed/final/X_train.csv`  
- `/data/processed/final/X_test.csv`  
- `/data/processed/final/y_train.csv`  
- `/data/processed/final/y_test.csv`

### Outputs
- Best model files: `/outputs/models/`  
- Evaluation metrics: `/outputs/metrics/`  
- Feature importances: `/outputs/ft_importance/`  
- SHAP values: `/outputs/shap_values/`

## Change Working Directory
- Since it is expected that you would keep the notebooks in a subfolder, you will need to switch the working directory when you run the notebook in the editor.
- The working directory must be changed from its current folder to its parent folder.
- We wish to change the current directory's parent to the new current directory.
- Verify the updated current directory.

In [ ]:
# Smart Working Directory Setup
import os
project_root = '/workspaces/heritage_housing'
if os.getcwd() != project_root:
    try:
        os.chdir(project_root)
        print(f"[INFO] Changed working directory to project root: {os.getcwd()}")
    except FileNotFoundError:
        raise FileNotFoundError(f"[ERROR] Project root '{project_root}' not found!")

### Load Libraries
- Load libraries
- Load cleaned data from /data/processed/df_cleaned.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib
import os

# Load processed data (engineered and encoded)
X_train = pd.read_csv("data/processed/final/X_train.csv")
X_test = pd.read_csv("data/processed/final/X_test.csv")
y_train = pd.read_csv("data/processed/final/y_train.csv").squeeze()
y_test = pd.read_csv("data/processed/final/y_test.csv").squeeze()

# Reconstruct df-like object for compatibility with rest of the notebook
df = pd.concat([X_train, y_train.rename("LogSalePrice")], axis=1)

print("[INFO] Final engineered data loaded successfully.")
print(f"[INFO] Dataset contains {df.shape[0]} rows and {df.shape[1]} columns.\n")

# Preview
print("[INFO] Sample of training dataset:")
display(df.head())

# Check for missing values
missing = df.isnull().sum()
missing = missing[missing > 0]

if not missing.empty:
    print("\n[WARNING] Missing values detected:")
    display(missing)
else:
    print("[INFO] No missing values found in the dataset.")


### Train-Test Split

**Objective:**

Split the cleaned dataset into training and testing subsets to evaluate model generalizability and avoid overfitting.

**Why Train-Test Split?**

- Prevents data leakage by ensuring evaluation is done on unseen data.
- Provides a realistic estimate of model performance before deployment.
- Essential for tracking overfitting or underfitting during experimentation.

In [ ]:
display(y_train.head())

# Confirm shape of the splits
print(f"[INFO] X_train shape: {X_train.shape}")
print(f"[INFO] X_test shape:  {X_test.shape}")
print(f"[INFO] y_train shape: {y_train.shape}")
print(f"[INFO] y_test shape:  {y_test.shape}")

# Preview training data
print("\n[INFO] Preview of X_train:")
display(X_train.head())

print("\n[INFO] Preview of y_train:")
display(y_train.head())


### Baseline Model Evaluation

Before diving into complex tuning or ensembling, it's crucial to evaluate some baseline regressors. This gives us a performance benchmark that future models must beat. It also helps identify underfitting or misalignment between features and the target variable.

**We’ll begin with:**

- Linear Regression – to test linearity assumptions
- Decision Tree Regressor – to capture non-linear patterns
- Random Forest Regressor – as a robust tree-based ensemble baseline

We’ll use R2, MAE, and RMSE for evaluation.

**Encode Categorical Variables**

In [ ]:
import pandas as pd
import numpy as np

# Load feature-engineered training and testing data
X_train = pd.read_csv("data/processed/final/X_train.csv")
X_test = pd.read_csv("data/processed/final/X_test.csv")
y_train = pd.read_csv("data/processed/final/y_train.csv").squeeze()
y_test = pd.read_csv("data/processed/final/y_test.csv").squeeze()

print("Feature-engineered data loaded successfully.\n")

# Dataset structure checks
print("Training Features Dataset Info:")
print(X_train.info())

print("\nTraining Target Stats:")
print(y_train.describe())

print("\nTesting Features Dataset Info:")
print(X_test.info())

print("\nTesting Target Stats:")
print(y_test.describe())

print("\nPreview of X_train:")
display(X_train.head())

print("\nPreview of X_test:")
display(X_test.head())


**Define Evaluation Function**

- To evaluate and compare multiple models quickly.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Baseline: predict the mean of y_train
y_baseline_train = [y_train.mean()] * len(y_train)
y_baseline_test = [y_train.mean()] * len(y_test)

# Evaluate metrics
baseline_metrics = {
    "Dataset": ["Train", "Test"],
    "R2": [
        r2_score(y_train, y_baseline_train),
        r2_score(y_test, y_baseline_test)
    ],
    "MAE": [
        mean_absolute_error(y_train, y_baseline_train),
        mean_absolute_error(y_test, y_baseline_test)
    ],
    "RMSE": [
        np.sqrt(mean_squared_error(y_train, y_baseline_train)),
        np.sqrt(mean_squared_error(y_test, y_baseline_test))
    ]
}

# Convert to DataFrame
baseline_df = pd.DataFrame(baseline_metrics)
print("Baseline Model Metrics:")
display(baseline_df)

# Visualize: predictions vs actual (test set)
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_baseline_test, alpha=0.6, label="Baseline Predictions")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Perfect Fit")
plt.title("Baseline Model: Predictions vs Actual (Test Set)")
plt.xlabel("Actual LogSalePrice")
plt.ylabel("Predicted LogSalePrice")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### Baseline Model Performance

#### Overview

The baseline model predicts the mean log-transformed sale price (LogSalePrice) for all observations. It doesn’t use any features — just averages — and serves as a reference point. Every meaningful model going forward must beat this benchmark.

**Results Summary**

| Dataset   | R2 Score |  MAE     | MSE 

| Training  | 0.00000  | 0.303367 | 0.152442

| Testing   | -0.00584 | 0.337137 | 0.187703

**Interpretation:**

- R2: As expected, the training R2 is 0.00 (predicts the mean), and testing R2 is negative — the model performs worse than just guessing the average.
- MAE: Around 0.30–0.34, which is significant. There’s room for improvement.
- MSE: Reinforces the poor performance — it penalizes larger errors more heavily.

#### Visualization Insights

**The scatterplot clearly shows:**

- Predictions are a flat horizontal line (the mean LogSalePrice).
- True values (Y-axis) vary widely, exposing the model’s inability to generalize.
- The dotted diagonal (perfect predictions) lies far from the flat baseline.

#### Limitations

- Ignores feature patterns.
- Offers no learning capability.
- Not meant for prediction — only for benchmarking.

### Next Steps

**Train advanced models:**

- Linear, Ridge, Lasso
- Decision Tree, Random Forest, Gradient Boosting, SVR
- Track the same metrics (R2, MAE, RMSE) for fair comparison.
- Use cross-validation and hyperparameter tuning to optimize.
- Visualize residuals and analyze feature importance.

## Model Training & Hyperparameter Tuning

### Objective:

Train multiple regression models to predict LogSalePrice and identify the best performer based on validation metrics. We'll use Grid Search for tuning where applicable, and visualize the results for comparison.

**Models Considered:**

We’ll train the following models to evaluate both linear and non-linear performance:

- `Linear Regression`: Simple baseline for linear trends
- `Ridge Regression`: Penalizes large coefficients (L2)
- `Lasso Regression`: Useful for feature selection (L1)
- `Decision Tree`: Handles non-linear splits, interpretable
- `Random Forest`: Ensemble of trees to reduce variance
- `Gradient Boosting Regressor`: Builds trees sequentially, optimizes performance
- `Support Vector Regressor`: Captures complex relationships, especially in high dimensions
- `K-Nearest Neighbors`: Instance-based learner for pattern matching

### Train a Variety of Models

This section will include:

- Use the training dataset to train many machine learning models.
- Use important metrics like R1, MAE, and MSE to compare how well they performed on the testing dataset.
- Determine which models perform best and merit further refinement in the following steps by visualizing and analyzing the data.
- Finding top-performing models to forecast home values based on important characteristics and associated hyperparameter setups is the goal.

In [ ]:
# === Import Required Libraries ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# === Reload Data with Encoding ===
X_train_raw = pd.read_csv("data/processed/final/X_train.csv")
X_test_raw = pd.read_csv("data/processed/final/X_test.csv")

# Identify categorical columns
categorical_cols = X_train_raw.select_dtypes(include="object").columns.tolist()

# One-hot encode categorical columns
X_train = pd.get_dummies(X_train_raw, columns=categorical_cols, drop_first=True)
X_test = pd.get_dummies(X_test_raw, columns=categorical_cols, drop_first=True)

# Align columns
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Load target
y_train = pd.read_csv("data/processed/final/y_train.csv").squeeze()
y_test = pd.read_csv("data/processed/final/y_test.csv").squeeze()

# === Define Evaluation Function ===
def evaluate_model(name, model, X_train, X_test, y_train, y_test, plot=True, verbose=True):
    """
    Trains a model and evaluates its performance on training and test data.

    Parameters:
        name (str): Name of the model.
        model (sklearn estimator): The machine learning model to evaluate.
        X_train (pd.DataFrame): Training features.
        X_test (pd.DataFrame): Test features.
        y_train (pd.Series): Training target.
        y_test (pd.Series): Test target.

    Returns:
        dict: Dictionary with model performance metrics (R2, MAE, RMSE).
    """
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    def get_metrics(y_true, y_pred):
        return {
            "R2": r2_score(y_true, y_pred),
            "MAE": mean_absolute_error(y_true, y_pred),
            "RMSE": np.sqrt(mean_squared_error(y_true, y_pred))
        }

    train_metrics = get_metrics(y_train, y_train_pred)
    test_metrics = get_metrics(y_test, y_test_pred)

    if verbose:
        print(f"\n{name} Performance:")
        print("Train:")
        for k, v in train_metrics.items():
            print(f"  {k}: {v:.4f}")
        print("Test:")
        for k, v in test_metrics.items():
            print(f"  {k}: {v:.4f}")

    if plot:
        residuals = y_test - y_test_pred
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        sns.histplot(residuals, kde=True)
        plt.title(f"{name} Residual Distribution")
        plt.xlabel("Residuals")

        plt.subplot(1, 2, 2)
        sns.scatterplot(x=y_test, y=y_test_pred, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
        plt.title(f"{name} Predicted vs Actual")
        plt.xlabel("Actual")
        plt.ylabel("Predicted")

        plt.tight_layout()
        plt.show()

    return {
        "Model": name,
        "Train R2": train_metrics["R2"],
        "Train MAE": train_metrics["MAE"],
        "Train RMSE": train_metrics["RMSE"],
        "Test R2": test_metrics["R2"],
        "Test MAE": test_metrics["MAE"],
        "Test RMSE": test_metrics["RMSE"]
    }


In [ ]:
# === Define and Evaluate Models ===
models = [
    ("Linear Regression", LinearRegression()),
    ("Decision Tree", DecisionTreeRegressor(random_state=42)),
    ("Random Forest", RandomForestRegressor(random_state=42)),
    ("Support Vector Regressor", SVR()),
    ("K-Nearest Neighbors", KNeighborsRegressor())
]

results = []
for name, model in models:
    result = evaluate_model(name, model, X_train, X_test, y_train, y_test)
    results.append(result)

# === Display Results Summary ===
results_df = pd.DataFrame(results)
print("\nModel Evaluation Summary:")
display(results_df)

#### Filter Out Extreme Models

If you're running multiple baselines or test runs, you may want to remove models with clearly poor R2 scores.

In [ ]:
print(results_df.columns)

# Filter out extreme underperformers
filtered_results_df = results_df[results_df["Test R2"] > 0]


In [ ]:
"""
Visual Comparison of Model Performance (Train vs Test)

This section plots R2, MAE, and RMSE for all models,
split by training and testing datasets, using bar charts.
Helps identify overfitting/underfitting and supports model selection.
"""

import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Create output directory if it doesn't exist
os.makedirs("outputs/visuals", exist_ok=True)

# Melt the DataFrame to long format
melted_df = results_df.melt(
    id_vars=["Model"],
    value_vars=["Train R2", "Test R2", "Train MAE", "Test MAE", "Train RMSE", "Test RMSE"],
    var_name="MetricType",
    value_name="Score"
)

# Split 'MetricType' into two columns: 'Metric' and 'Set'
melted_df[["Set", "Metric"]] = melted_df["MetricType"].str.split(" ", expand=True)
melted_df.drop("MetricType", axis=1, inplace=True)

# Set aesthetics
sns.set_theme(style="whitegrid")

# Plotting loop for each metric
for metric in ["R2", "MAE", "RMSE"]:
    plt.figure(figsize=(10, 6))
    metric_df = melted_df[melted_df["Metric"] == metric]
    sns.barplot(data=metric_df, x="Score", y="Model", hue="Set", palette="Set2")
    plt.title(f"{metric} Score Comparison: Training vs Testing")
    plt.xlabel(metric)
    plt.ylabel("Model")
    plt.legend(title="Dataset", loc="best")
    plt.tight_layout()

    # Save each plot
    filepath = f"outputs/visuals/{metric.lower()}_score_comparison.png"
    plt.savefig(filepath)
    plt.show()


#### Observations

**Performance Overview:** 
- The testing dataset's highest R2 score (0.81) was attained by the Random Forest model, suggesting that it well captures the target variable.
- The Decision Tree model demonstrated signs of overfitting with a lower testing R2 (0.74), but it did remarkably well on the training dataset (R2: 0.999).
- With low R2 values for testing, both Linear Regression and Support Vector Regressor fared poorly, suggesting poor generalization.

**Suitability for Additional adjustment:** 
- Because of their comparatively good testing results, models like Random Forest and Decision Tree make excellent candidates for hyperparameter adjustment.
- To increase performance, models like SVR and K-Nearest Neighbours might need to be adjusted.

### Hyperparameter Tuning

#### Objectives:
To enhance the performance of the best-performing models on the testing dataset, this part attempts to adjust their hyperparameters.

**Models Chosen for Adjustment**
- Random Forest
- Decision Tree
- Gradient Boosting
- Ridge
- SVR (Support Vector) 

#### Random Forest Regressor

In [ ]:
"""
Random Forest Hyperparameter Tuning using GridSearchCV

This block performs exhaustive search across specified hyperparameter combinations
for a Random Forest Regressor to optimize model performance using 5-fold cross-validation.
Evaluated using R2 score.
"""

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# --- Define hyperparameter grid ---
param_grid_rf = {
    "n_estimators": [100, 200],         # Number of trees in the forest
    "max_depth": [10, 20, None],        # Tree depth control
    "min_samples_split": [2, 5],        # Minimum samples required to split an internal node
    "min_samples_leaf": [1, 2],         # Minimum samples required at each leaf node
}

# --- Initialize GridSearchCV ---
rf_grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid_rf,
    cv=5,
    scoring="r2",
    verbose=2,
    n_jobs=-1
)

# --- Execute grid search ---
rf_grid_search.fit(X_train, y_train)

# --- Output best parameters and cross-validated R2 score ---
print("Best Parameters for Random Forest:", rf_grid_search.best_params_)
print("Best R2-Score for Random Forest:", rf_grid_search.best_score_)


**Evaluate Best Random Forest Model on Test Data**

- This will use the best parameters found during GridSearchCV to make predictions on the test set and compute the R2, MAE, and RMSE.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Use the best estimator from GridSearchCV
best_rf_model = rf_grid_search.best_estimator_

# Predict on the test set
rf_test_preds = best_rf_model.predict(X_test)

# Evaluate performance
r2 = r2_score(y_test, rf_test_preds)
mae = mean_absolute_error(y_test, rf_test_preds)
rmse = np.sqrt(mean_squared_error(y_test, rf_test_preds))

print("Tuned Random Forest Test Set Performance:")
print(f"R2 Score: {r2:.4f}")
print(f"MAE:      {mae:.4f}")
print(f"RMSE:     {rmse:.4f}")


**Visualize Model Performance**

In [ ]:
# Extract feature importances
importances = best_rf_model.feature_importances_
feature_names = X_train.columns

# Create a DataFrame for visualization
importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

# Plot top 15 important features
plt.figure(figsize=(10, 8))
sns.barplot(data=importance_df.head(15), x="Importance", hue="Feature", y="Feature", palette="viridis", legend=False)
plt.title("Top 15 Feature Importances - Random Forest Regressor")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.tight_layout()

# Save the plot
plt.savefig("outputs/visuals/rf_feature_importance_top15.png")
plt.show()

### Decision Tree

In [ ]:
# Load necessary libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import os

# === Reload Data with Encoding ===
X_train_raw = pd.read_csv("data/processed/final/X_train.csv")
X_test_raw = pd.read_csv("data/processed/final/X_test.csv")

# Identify categorical columns
categorical_cols = X_train_raw.select_dtypes(include="object").columns.tolist()

# One-hot encode
X_train = pd.get_dummies(X_train_raw, columns=categorical_cols, drop_first=True)
X_test = pd.get_dummies(X_test_raw, columns=categorical_cols, drop_first=True)

# Align columns to ensure matching features
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Load target
y_train = pd.read_csv("data/processed/final/y_train.csv").squeeze()
y_test = pd.read_csv("data/processed/final/y_test.csv").squeeze()


In [ ]:
# Define parameter grid for Decision Tree
dt_param_grid = {
    "max_depth": [3, 5, 10, 15, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

# Grid search with cross-validation
dt_grid_search = GridSearchCV(
    estimator=DecisionTreeRegressor(random_state=42),
    param_grid=dt_param_grid,
    cv=5,
    scoring="r2",
    verbose=1,
    n_jobs=-1
)

# Fit model
dt_grid_search.fit(X_train, y_train)

# Best model and parameters
best_dt_model = dt_grid_search.best_estimator_
best_dt_params = dt_grid_search.best_params_
best_dt_score = dt_grid_search.best_score_

print("Best Decision Tree Parameters:", best_dt_params)
print("Best CV R2 Score:", round(best_dt_score, 4))



**Evaluate Tuned Decision Tree Performance on Test Data**

In [ ]:
# Predict on the test set
y_pred_dt = best_dt_model.predict(X_test)

# Calculate evaluation metrics
dt_r2 = r2_score(y_test, y_pred_dt)
dt_mae = mean_absolute_error(y_test, y_pred_dt)
dt_rmse = np.sqrt(mean_squared_error(y_test, y_pred_dt))

# Display performance
print("Tuned Decision Tree Performance on Test Data:")
print(f"R2 Score:   {dt_r2:.4f}")
print(f"MAE:        {dt_mae:.4f}")
print(f"RMSE:       {dt_rmse:.4f}")


**Visualize Model Performance**

In [ ]:
# Test evaluation
y_pred_dt = best_dt_model.predict(X_test)
dt_r2 = r2_score(y_test, y_pred_dt)
dt_mae = mean_absolute_error(y_test, y_pred_dt)
dt_rmse = np.sqrt(mean_squared_error(y_test, y_pred_dt))

print("\nTuned Decision Tree Performance on Test Data:")
print(f"R2 Score: {dt_r2:.4f}")
print(f"MAE:      {dt_mae:.4f}")
print(f"RMSE:     {dt_rmse:.4f}")

# Visualize top 15 features
dt_feature_importances = best_dt_model.feature_importances_
dt_features = X_train.columns
dt_importance_df = pd.DataFrame({
    "Feature": dt_features,
    "Importance": dt_feature_importances
}).sort_values(by="Importance", ascending=False)

# Plot
plt.figure(figsize=(10, 8))
sns.barplot(data=dt_importance_df.head(15), x="Importance", y="Feature", palette="viridis", hue="Feature", legend=False)
plt.title("Top 15 Feature Importances - Decision Tree Regressor")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.tight_layout()
os.makedirs("outputs/visuals", exist_ok=True)
plt.savefig("outputs/visuals/dt_feature_importance_top15.png")
plt.show()


### Gradient Boosting Regressor

In [ ]:
"""
Gradient Boosting Regressor - Hyperparameter Tuning with GridSearchCV

This block searches for the best combination of hyperparameters for the Gradient Boosting model 
to maximize R2 performance via 5-fold cross-validation.
"""

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# --- Define hyperparameter grid ---
gbr_param_grid = {
    "n_estimators": [100, 200],          # Number of boosting stages
    "max_depth": [3, 4, 5],              # Max depth of each individual tree
    "learning_rate": [0.01, 0.05, 0.1],  # Step size shrinkage
    "min_samples_split": [2, 5],         # Min samples required to split a node
    "min_samples_leaf": [1, 2]           # Min samples at a leaf node
}

# --- Initialize GridSearchCV ---
gbr_grid_search = GridSearchCV(
    estimator=GradientBoostingRegressor(random_state=42),
    param_grid=gbr_param_grid,
    cv=5,
    scoring="r2",
    verbose=2,
    n_jobs=-1
)

# --- Execute grid search ---
gbr_grid_search.fit(X_train, y_train)

# --- Output best parameters and cross-validation R2 score ---
print("Best Parameters for Gradient Boosting:", gbr_grid_search.best_params_)
print("Best R2 Score for Gradient Boosting:", gbr_grid_search.best_score_)


**Evaluate Tunes Gradient Boosting on Test Data**

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Use the best estimator from GridSearchCV
best_gbr_model = gbr_grid_search.best_estimator_

# Predict on the test set
gbr_test_preds = best_gbr_model.predict(X_test)

# Evaluate performance
r2 = r2_score(y_test, gbr_test_preds)
mae = mean_absolute_error(y_test, gbr_test_preds)
rmse = np.sqrt(mean_squared_error(y_test, gbr_test_preds))

print("Tuned Gradient Boosting Test Set Performance:")
print(f"R2 Score: {r2:.4f}")
print(f"MAE:      {mae:.4f}")
print(f"RMSE:     {rmse:.4f}")


**Visualize Model Performance**

In [ ]:
# Get the best estimator from grid search
best_gbr_model = gbr_grid_search.best_estimator_

# Extract feature importances
gbr_importances = best_gbr_model.feature_importances_
feature_names = X_train.columns

# Create DataFrame
gbr_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": gbr_importances
}).sort_values(by="Importance", ascending=False)

# Plot top 15 features
plt.figure(figsize=(10, 8))
sns.barplot(data=gbr_importance_df.head(15), x="Importance", y="Feature", palette="viridis", hue="Feature", legend=False)
plt.title("Top 15 Feature Importances - Gradient Boosting Regressor")
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.tight_layout()

# Save the plot
plt.savefig("outputs/visuals/gbr_feature_importance_top15.png")
plt.show()


### Ridge Regression

In [ ]:
"""
Ridge Regression - Hyperparameter Tuning with GridSearchCV

This step performs an exhaustive grid search across different alpha values and solvers 
to identify the best regularized linear model using 5-fold cross-validation.
"""

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# --- Define hyperparameter grid ---
ridge_param_grid = {
    "alpha": [0.01, 0.1, 1, 10, 100],              # Regularization strength
    "solver": ["auto", "svd", "cholesky", "lsqr", "sag"]  # Different algorithms for optimization
}

# --- Initialize GridSearchCV ---
ridge_grid_search = GridSearchCV(
    estimator=Ridge(random_state=42),
    param_grid=ridge_param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1,
    verbose=1
)

# --- Execute grid search ---
ridge_grid_search.fit(X_train, y_train)

# --- Output best parameters and cross-validation score ---
print("Best Parameters for Ridge Regression:", ridge_grid_search.best_params_)
print("Best R2 Score for Ridge Regression:", ridge_grid_search.best_score_)


**Evaluate Tunes Ridge Regression Performance on Test Data**

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Retrieve the best Ridge model
best_ridge_model = ridge_grid_search.best_estimator_

# Predict on test data
y_pred_ridge = best_ridge_model.predict(X_test)

# Calculate evaluation metrics
ridge_r2 = r2_score(y_test, y_pred_ridge)
ridge_mae = mean_absolute_error(y_test, y_pred_ridge)
ridge_mse = mean_squared_error(y_test, y_pred_ridge)

# Display results
print("Tuned Ridge Regression Performance on Test Data:")
print(f"R2 Score:   {ridge_r2:.4f}")
print(f"MAE:        {ridge_mae:.4f}")
print(f"MSE:        {ridge_mse:.4f}")

**Visualize Model Performance**

In [ ]:
# Extract coefficients
ridge_coef = best_ridge_model.coef_
feature_names = X_train.columns

# Create DataFrame for visualization
ridge_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": np.abs(ridge_coef)
}).sort_values(by="Importance", ascending=False)

# Plot top 15 features
plt.figure(figsize=(10, 8))
sns.barplot(data=ridge_importance_df.head(15), x="Importance", y="Feature", palette="viridis", hue="Feature", legend=False)
plt.title("Top 15 Feature Importances - Ridge Regressor (via Coefficients)")
plt.xlabel("Absolute Coefficient")
plt.ylabel("Feature")
plt.tight_layout()

# Save the plot
plt.savefig("outputs/visuals/rr_feature_importance_top15.png")
plt.show()


### Support Vector Regression(SVR)

In [ ]:
"""
Support Vector Regressor - Hyperparameter Tuning with GridSearchCV

This step tunes the SVR model by testing combinations of kernel parameters (C, gamma, epsilon)
to capture complex relationships in housing data. 5-fold cross-validation ensures stability.
"""

from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# --- Define hyperparameter grid ---
svr_param_grid = {
    "kernel": ["rbf"],              # Radial basis function kernel for non-linear mapping
    "C": [1, 10, 100],              # Regularization parameter
    "gamma": ["scale", "auto"],    # Kernel coefficient
    "epsilon": [0.1, 0.2, 0.5]      # Insensitivity within margin
}

# --- Initialize GridSearchCV ---
svr_grid_search = GridSearchCV(
    estimator=SVR(),
    param_grid=svr_param_grid,
    scoring="r2",
    cv=5,
    verbose=2,
    n_jobs=-1
)

# --- Execute grid search ---
svr_grid_search.fit(X_train, y_train)

# --- Output best configuration and performance ---
print("Best Parameters for SVR:", svr_grid_search.best_params_)
print("Best R2 Score from Grid Search (CV):", svr_grid_search.best_score_)


**Evaluate Tuned SVR Model on Test Data**

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Retrieve the best SVR model
best_svr_model = svr_grid_search.best_estimator_

# Predict on test data
y_pred_svr = best_svr_model.predict(X_test)

# Calculate metrics
svr_r2 = r2_score(y_test, y_pred_svr)
svr_mae = mean_absolute_error(y_test, y_pred_svr)
svr_mse = mean_squared_error(y_test, y_pred_svr)

# Display results
print("Tuned SVR Performance on Test Data:")
print(f"R2 Score:   {svr_r2:.4f}")
print(f"MAE:        {svr_mae:.4f}")
print(f"MSE:        {svr_mse:.4f}")


**Visualize Model Performance**

In [ ]:
from sklearn.inspection import permutation_importance

# Compute permutation importance on SVR
svr_perm = permutation_importance(best_svr_model, X_test, y_test, n_repeats=30, random_state=42, n_jobs=-1)

# Create DataFrame
svr_importance_df = pd.DataFrame({
    "Feature": X_test.columns,
    "Importance": svr_perm.importances_mean
}).sort_values(by="Importance", ascending=False)

# Plot top 15
plt.figure(figsize=(10, 8))
sns.barplot(data=svr_importance_df.head(15), x="Importance", y="Feature", palette="viridis", hue="Feature", legend=False)
plt.title("Top 15 Feature Importances - SVR (via Permutation Importance)")
plt.xlabel("Mean Importance Score")
plt.ylabel("Feature")
plt.tight_layout()

# Save the plot
plt.savefig("outputs/visuals/svr_feature_importance_top15.png")
plt.show()


### Tuned Model Performance Observations (Test Set)
 
**Random Forest Regressor**

- R2 Score: 0.8766 — Excellent predictive strength, explains nearly 88% of variance.
- MAE: 0.1018 — Low average prediction error.
- RMSE: 0.1517 — Indicates strong performance in penalizing large errors.

Observation: A top performer — accurate, robust, and well-balanced. Ideal for deployment if interpretability isn’t the top priority.

**Decision Tree Regressor**

- R2 Score: 0.8058 — Decent performance, but lower than ensemble models.
- MAE: 0.1373 — Higher error compared to others.
- MSE: 0.0362 — Acceptable but suggests overfitting risk.
  
Observation: Useful for explainability, but lacks generalization. Not ideal for final deployment alone.

**Gradient Boosting Regressor**

- R2 Score: 0.8774 — Best-in-class R2, slightly edging out Random Forest.
- MAE: 0.0998 — Lowest mean error across all models.
- RMSE: 0.1513 — Tight error distribution, very stable.

Observation: Highest overall performance — suitable for final deployment if latency isn’t a concern. Performs especially well on structured/tabular data.

**Ridge Regression**

- R2 Score: 0.8686 — Solid performance for a regularized linear model.
- MAE: 0.1081 — Slightly higher than tree-based models.
- MSE: 0.0245 — Very low squared error.

Observation: A strong linear baseline. Efficient, interpretable, and reliable for situations needing quick or explainable predictions.

**Support Vector Regressor (SVR)**

- R2 Score: 0.7908 — Lowest R2 of the tuned models.
- MAE: 0.1189 — Reasonable average error.
- MSE: 0.0390 — Moderate squared error.

Observation: Performs adequately, but lags behind boosted and ensemble methods. High tuning complexity with limited marginal gain.

## Model Comparison & Consolidation

### Objective

The goal of this section is to consolidate the performance results from all tuned regression models and visualize them side-by-side using R2, MAE, and RMSE. This comparison provides clarity on which models generalize best to unseen data and serve as the most viable candidates for deployment.

We assess each model's predictive power, accuracy, and robustness by evaluating:

- R2 Score: How well the model explains variance in the target variable.
- MAE: The average magnitude of errors.
- RMSE: The standard deviation of residuals, penalizing large errors.

In [ ]:
# Import necessary libraries after kernel reset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Consolidated model results
model_results = [
    {"Model": "Random Forest", "R2": 0.8766, "MAE": 0.1018, "RMSE": 0.1517},
    {"Model": "Decision Tree", "R2": dt_r2, "MAE": dt_mae, "RMSE": dt_rmse},
    {"Model": "Gradient Boosting", "R2": 0.8774, "MAE": 0.0998, "RMSE": 0.1513},
    {"Model": "Ridge Regression", "R2": 0.8686, "MAE": 0.1081, "RMSE": 0.1565},
    {"Model": "Support Vector Regressor", "R2": 0.7908, "MAE": 0.1189, "RMSE": 0.1975}
]

results_df = pd.DataFrame(model_results)

# Plotting
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# R2 Plot
sns.barplot(data=results_df, x="R2", y="Model", palette="crest", ax=axes[0], hue="Model")
axes[0].set_title("Model Comparison: R2 Score")
axes[0].set_xlim(0.75, 0.90)

# MAE Plot
sns.barplot(data=results_df, x="MAE", y="Model", palette="flare", ax=axes[1], hue="Model")
axes[1].set_title("Model Comparison: Mean Absolute Error (MAE)")

# RMSE Plot
sns.barplot(data=results_df, x="RMSE", y="Model", palette="mako", ax=axes[2], hue="Model")
axes[2].set_title("Model Comparison: Root Mean Squared Error (RMSE)")

plt.tight_layout()

# Save the full figure
plot_path = "outputs/visuals/final_model_comparison.png"
plt.savefig(plot_path)
plt.show()

print("Consolidated Model Results:")
display(results_df)
print(f"[SAVED] Model comparison plot saved to: {plot_path}")


### Observations:
- Gradient Boosting Regressor slightly outperformed others with the highest R2 (0.8774) and lowest RMSE (0.1513), making it the strongest candidate overall.
- Random Forest Regressor came very close in performance, offering excellent results with slightly higher RMSE than Gradient Boosting.
- Ridge Regression provided strong linear performance (R2: 0.8686), indicating regularized linear models can still compete effectively when features are well-prepared.
- Decision Tree Regressor performed reasonably well but showed signs of overfitting compared to ensemble methods.
- SVR had the lowest R2 (0.7908), indicating it was less suited to this dataset under current hyperparameters.

**Conclusion:**

Ensemble models like Gradient Boosting and Random Forest consistently outperformed other methods and are recommended for final deployment.

## Cross-Validation Results (5-Fold)

To ensure model robustness and assess generalizability, we performed 5-fold cross-validation on each of the tuned models using the training data.

This process helps validate whether the model is overfitting or generalizing well before we evaluate it on the test set.

The evaluation metrics include:
- **R2 Score**: Measures how well the model explains the variance.
- **MAE**: Mean Absolute Error (lower is better).
- **RMSE**: Root Mean Squared Error (penalizes large errors).

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# Define scoring functions
scorers = {
    "R2": "r2",
    "MAE": make_scorer(mean_absolute_error),
    "RMSE": make_scorer(mean_squared_error, squared=False)
}

# Dictionary of tuned models
tuned_models = {
    "Random Forest": best_rf_model,
    "Decision Tree": best_dt_model,
    "Gradient Boosting": best_gbr_model,
    "Ridge Regression": best_ridge_model,
    "SVR": best_svr_model
}

cv_results = []

# Evaluate each tuned model with 5-fold CV
for name, model in tuned_models.items():
    r2 = cross_val_score(model, X_train, y_train, cv=5, scoring=scorers["R2"]).mean()
    mae = -cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_absolute_error").mean()
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_squared_error").mean())

    cv_results.append({
        "Model": name,
        "CV R2": round(r2, 4),
        "CV MAE": round(mae, 4),
        "CV RMSE": round(rmse, 4)
    })

# Convert to DataFrame
cv_results_df = pd.DataFrame(cv_results).sort_values(by="CV R2", ascending=False)

# Display
print("Cross-Validation Results:")
display(cv_results_df)

# Set up subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot R2
sns.barplot(data=cv_results_df, x="CV R2", y="Model", ax=axes[0], palette="crest", hue="Model", legend=False)
axes[0].set_title("Cross-Validated R2 Scores")

# Plot MAE
sns.barplot(data=cv_results_df, x="CV MAE", y="Model", ax=axes[1], palette="flare", hue="Model", legend=False)
axes[1].set_title("Cross-Validated MAE")

# Plot RMSE
sns.barplot(data=cv_results_df, x="CV RMSE", y="Model", ax=axes[2], palette="mako", hue="Model", legend=False)
axes[2].set_title("Cross-Validated RMSE")

plt.tight_layout()

# Save the figure
plt.savefig("outputs/visuals/cross_validation_results.png")
plt.show()

### Observations:

**Gradient Boosting Regressor demonstrated the strongest overall performance across all metrics:**

- Highest CV R2 (0.8756), indicating excellent ability to explain variance in house prices.
- Lowest MAE (0.0951) and RMSE (0.1376), suggesting precise and consistent predictions.

**Random Forest Regressor was a close second:**

- CV R2 of 0.8581 and slightly higher error metrics compared to Gradient Boosting.
- A strong, stable model suitable for general use cases where interpretability is less critical.

**Ridge Regression performed well for a linear model:**

- Achieved CV R2 of 0.8176 with relatively low error.
- Its simplicity and resistance to overfitting make it valuable when linearity is assumed or preferred.

**Support Vector Regressor (SVR) showed moderate predictive strength:**

- Slightly lower CV R2 at 0.7974 and higher MAE/RMSE compared to ensemble models.
- Could be improved with feature scaling and advanced kernel tuning.

**Decision Tree Regressor was the weakest performer:**

- CV R2 of 0.7156, with the highest MAE (0.1470) and RMSE (0.2071).
- Prone to overfitting and instability, making it less reliable without pruning or ensembling.

## Test Set Evaluation

### Objective

The goal of this section is to evaluate each tuned model's performance on the unseen test set. This helps determine how well the models generalize beyond the training data. By comparing these scores to the cross-validation results, we assess model stability, overfitting risk, and final performance readiness for deployment.

We will:
- Consolidate the test set R2, MAE, and RMSE for each tuned model.
- Compare them across models.
- Provide a visual summary for quick interpretation.
- Highlight any overfitting or underfitting behavior.

In [ ]:
# Re-import necessary libraries after kernel reset
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Define the test results dictionary
test_results = {
    "Model": [
        "Gradient Boosting", 
        "Random Forest", 
        "Ridge Regression", 
        "SVR", 
        "Decision Tree"
    ],
    "Test R2": [
        0.8774, 0.8766, 0.8686, 0.7908, 0.8058
    ],
    "Test MAE": [
        0.0998, 0.1018, 0.1081, 0.1189, 0.1373
    ],
    "Test RMSE": [
        0.1513, 0.1517, 0.1565, 0.1975, 0.1903
    ]
}

# Create DataFrame
test_results_df = pd.DataFrame(test_results)
test_results_df_sorted = test_results_df.sort_values(by="Test R2", ascending=False)

# Plot comparison for R2, MAE, and RMSE
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
sns.barplot(data=test_results_df_sorted, y="Model", x="Test R2", ax=axes[0], palette="crest", hue="Model", legend=False)
axes[0].set_title("Test Set R2 Scores")
axes[0].set_xlim(0, 1)

sns.barplot(data=test_results_df_sorted, y="Model", x="Test MAE", ax=axes[1], palette="flare", hue="Model", legend=False)
axes[1].set_title("Test Set MAE Scores")

sns.barplot(data=test_results_df_sorted, y="Model", x="Test RMSE", ax=axes[2], palette="magma", hue="Model", legend=False)
axes[2].set_title("Test Set RMSE Scores")

plt.tight_layout()

# Save the figure
plt.savefig("outputs/visuals/test_set_model_comparison.png")
plt.show()

### Observations:

- Gradient Boosting and Random Forest are top performers with R2 scores close to 0.88, indicating excellent model fit and generalization on unseen data.
- Gradient Boosting has the lowest MAE (0.0998) and RMSE (0.1513), making it the most accurate and consistent model across all metrics.
- Ridge Regression also performs robustly with a high R2 (0.8686), and its lower MAE and RMSE confirm its strong linear modeling capability.
- SVR and Decision Tree trail behind in all three metrics, with higher errors and comparatively lower R2 values, suggesting limited capacity to generalize complex price patterns.

**Recommendation:** 

- Gradient Boosting should be the preferred model moving forward for deployment.
- The Gradient Boosting model achieved an R2 of 0.8774 on unseen test data, clearly meeting the project’s performance goal of R2 ≥ 0.75 and validating its suitability for final deployment.
- These results confirm that the selected model generalizes well to unseen data and meets the original business objective of predicting fair property prices with confidence.

## Residual Analysis

### Objective

- Residual analysis helps assess the performance of regression models by visualizing and interpreting the difference between predicted and actual values (residuals). This is crucial to:
- Identify non-random patterns or bias.
- Detect heteroscedasticity (changing variance across predictions).
- Ensure model assumptions (e.g., normality, independence) are not violated.
- Spot influential outliers that might distort model predictions.

In [ ]:
# Dictionary of tuned models
tuned_models = {
    "Random Forest": best_rf_model,
    "Decision Tree": best_dt_model,
    "Gradient Boosting": best_gbr_model,
    "Ridge Regression": best_ridge_model,
    "SVR": best_svr_model
}

# Plot residual distributions
plt.figure(figsize=(14, 10))

for idx, (name, model) in enumerate(tuned_models.items(), 1):
    y_pred = model.predict(X_test)
    residuals = y_test - y_pred

    plt.subplot(3, 2, idx)
    sns.histplot(residuals, kde=True, bins=30)
    plt.title(f"Residual Distribution - {name}")
    plt.xlabel("Residuals")
    plt.ylabel("Frequency")

plt.tight_layout()

# Save the figure
plt.savefig("outputs/visuals/residual_distribution.png")
plt.show()


**Plot: Residuals vs. Predicted**

In [ ]:
# Plot residuals vs predicted values
plt.figure(figsize=(14, 10))

for idx, (name, model) in enumerate(tuned_models.items(), 1):
    y_pred = model.predict(X_test)
    residuals = y_test - y_pred

    plt.subplot(3, 2, idx)
    sns.scatterplot(x=y_pred, y=residuals, alpha=0.6)
    plt.axhline(0, linestyle='--', color='black', linewidth=1)
    plt.title(f"Residuals vs Predicted - {name}")
    plt.xlabel("Predicted Log(SalePrice)")
    plt.ylabel("Residuals")

plt.tight_layout()

# Save the figure
plt.savefig("outputs/visuals/residual_vs_predicted.png")
plt.show()

### Observations

- Random Forest & Gradient Boosting showed tight, symmetrical residual distributions with minimal bias — validating their superior R2 and RMSE scores.
- Ridge Regression produced reasonably centered residuals but had slightly longer tails, suggesting some underfitting on complex patterns.
- Decision Tree had higher error variance and showed signs of overfitting with outliers.
- SVR had wider scatter and some bias near the prediction extremes, confirming its slightly lower test R2.

## Feature Importance & Insights

### Objective

In this section, we aim to identify which features have the greatest influence on house price predictions. Understanding feature importance helps:

- Interpret model decisions
- Guide further feature engineering
- Offer actionable insights for stakeholders (e.g., real estate clients)

We will analyze feature importance from Random Forest and Decision Tree models — both tree-based models that naturally compute feature contribution based on information gain.

We chose these models because they are:

- Intrinsically interpretable
- Robust for mixed feature types
- Aligned with the structure and complexity of real estate data


**Random Forest Regressor: Feature Importance**

We reuse the previously generated feature importance plot for the Random Forest model to anchor the comparative insights in this section.

In [ ]:
from IPython.display import Image, display

print("Top 15 Feature Importances - Random Forest (Reused)")
display(Image("outputs/visuals/rf_feature_importance_top15.png"))


**Decision Tree Regressor: Feature Importance**

We reference the previously saved plot to avoid redundancy. The Decision Tree model highlights different splits, providing contrast to the Random Forest.


In [ ]:
from IPython.display import Image, display

print("Top 15 Feature Importances - Decision Tree (Reused)")
display(Image("outputs/visuals/dt_feature_importance_top15.png"))


#### Observations

- Overall Quality (num__OverallQual) ranked highest in both Random Forest and Decision Tree models, reinforcing its critical influence on sale prices.
- Above Ground Living Area (num__GrLivArea) and Garage Area (num__GarageArea) consistently showed high importance — supporting earlier correlation and residual analyses.
- Features like Finished Basement Ratio, Lot Area, and YearBuilt had medium to high importance, highlighting structure age and layout as price influencers.
- Low-importance features may still hold interaction effects (to be tested via SHAP/partial dependence next).

## Model Explainability

### (Random Forest) Objective

The goal of this section is to understand how the Random Forest model arrives at its predictions. This builds transparency and trust in the model’s behavior — especially for non-technical stakeholders. 

**We’ll focus on:**

- Identifying the most influential features globally.
- Understanding feature impact on individual predictions using SHAP (SHapley Additive exPlanations).
- Random Forests support intrinsic feature importance, and SHAP adds explainability at the individual prediction level.

**We will:**

- Revisit and visualize feature importances for both models
- Apply SHAP (SHapley Additive exPlanations) for deeper interpretability:
- Global interpretation: Understand which features drive predictions overall
- Local interpretation: Understand why the model made a specific prediction

**Import Model**

In [ ]:
%pip install shap

**SHAP Summary Plot for Random Forest (Global Interpretability)**

In [ ]:
"""
SHAP Analysis for Random Forest Model

This script calculates and visualizes SHAP (SHapley Additive exPlanations) values
to explain the feature contributions for predictions made by the tuned Random Forest model.
"""

import shap
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# --- Check and fill missing values in test/train sets ---
print("Checking for nulls in input datasets (Random Forest SHAP)...")
print("X_train null values:\n", X_train.isnull().sum().sort_values(ascending=False).head())

# Use mean imputation for missing numeric values to avoid SHAP errors
X_train_filled = X_train.fillna(X_train.mean())
X_test_filled = X_test.fillna(X_train.mean())  # use training mean to prevent data leakage

# --- Initialize Tree Explainer for Random Forest ---
print("Initializing SHAP explainer (Random Forest)...")
explainer_rf = shap.TreeExplainer(best_rf_model)

# --- Calculate SHAP values on test set ---
print("Calculating SHAP values for X_test (Random Forest)...")
shap_values_rf = explainer_rf.shap_values(X_test_filled)

# --- Summary Plot: Mean absolute SHAP values per feature ---
print("Plotting SHAP summary plot (bar)...")
shap.summary_plot(shap_values_rf, X_test_filled, plot_type="bar", show=True)

# --- Summary Plot: Detailed dot plot of SHAP values ---
print("Plotting SHAP summary plot (dot)...")
shap.summary_plot(shap_values_rf, X_test_filled, show=True)

# --- Create SHAP contribution table ---
print("Computing average SHAP contributions...")
shap_rf_df = pd.DataFrame(shap_values_rf, columns=X_test_filled.columns)
shap_rf_mean = shap_rf_df.abs().mean(axis=0).sort_values(ascending=False)
shap_rf_table = pd.DataFrame({
    "Feature": shap_rf_mean.index,
    "Mean SHAP Value": shap_rf_mean.values
})

# Display top contributing features
print("Top 10 Features by SHAP Contribution:")
print(shap_rf_table.head(10))

# --- Generate SHAP dependence plot for top feature ---
print("Generating SHAP dependence plot for top feature...")
top_feature = shap_rf_table["Feature"].iloc[0]
shap.dependence_plot(
    top_feature,
    shap_values_rf,
    X_test_filled,
    interaction_index=None,
    show=True
)

# Save dependence plot
plt.gcf().tight_layout()
plt.savefig(f"outputs/visuals/shap_rf_dependence_{top_feature}.png", bbox_inches="tight")


### (XGBoost) Objective

To understand how XGBoost makes predictions by analyzing feature-level contributions using SHAP (SHapley Additive exPlanations). This enhances model transparency and supports interpretability.

In [ ]:
"""
SHAP Explainability for Gradient Boosting Regressor (GBR)

This script uses SHAP (SHapley Additive exPlanations) to evaluate how each feature 
influences the model’s predictions. It generates feature importance tables, global summary plots, 
and a dependence plot for the most influential feature.
"""

import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Initialize SHAP Explainer for Gradient Boosting ---
explainer_gbr = shap.Explainer(best_gbr_model)

# --- Compute SHAP values on the test set ---
shap_values_gbr = explainer_gbr(X_test_filled)

# --- Mean absolute SHAP value per feature for ranking ---
shap_mean_gbr = np.abs(shap_values_gbr.values).mean(axis=0)

# --- Create DataFrame with SHAP importance ---
shap_importance_gbr_df = pd.DataFrame({
    "Feature": X_test_filled.columns,
    "Mean SHAP Value": shap_mean_gbr
}).sort_values(by="Mean SHAP Value", ascending=False)

# --- Display Top 10 Features ---
print("Top 10 Contributing Features (XGBoost SHAP):")
display(shap_importance_gbr_df.head(10))

# --- SHAP Summary Plots ---
print("Generating SHAP summary plots (bar and beeswarm)...")
shap.summary_plot(shap_values_gbr, X_test_filled, plot_type="bar", show=True)
shap.summary_plot(shap_values_gbr, X_test_filled, show=True)

# --- Dependence Plot for Top Feature ---
print("Generating SHAP dependence plot for top-ranked feature...")
top_feature_gbr = X_test_filled.columns[np.argsort(shap_mean_gbr)[-1]]  # top feature by SHAP

shap.dependence_plot(
    top_feature_gbr,
    shap_values_gbr.values,
    X_test_filled,
    interaction_index=None,
    show=True
)

# Save the dependence plot
plt.gcf().tight_layout()
plt.savefig(f"outputs/visuals/shap_gbr_dependence_{top_feature_gbr}.png", bbox_inches="tight")


#### SHAP Feature Contribution Observations:

The SHAP value analysis for both XGBoost and Random Forest reveals strong consensus on the most influential features driving house price predictions. Here's a breakdown of key observations:

**Common Top Contributors**

1. overallqual (Overall Quality):

- Ranked #1 in both models.
- It shows the highest SHAP value across the board, confirming it's the most decisive factor in predicting sale price.

Observation: Property quality is perceived and priced as a premium attribute, reaffirming earlier correlation and importance plots.

2. grlivarea (Above-Ground Living Area):

- Consistently ranked #2, with strong SHAP values.

Observation: Larger, usable living areas have direct positive impact on value perception and final pricing.

3. totalbsmtsf, bsmtfinsf1 (Basement Area & Finish):

- Appear in top 5 in both models.

Observation: Finished and spacious basements add perceived livability and investment value.

4. yearbuilt & yearremodadd:

- Reflects recency of construction or upgrades, appearing in top 10 consistently.

Observation: Modern or recently renovated homes command higher prices due to reduced maintenance risk and updated features.

5. garagearea:

- Present in both lists.

Observation: Functional garages are a valuable utility space, especially in suburban residential markets.

**Model-Specific Insights**

- Random Forest gives slightly more weight to overallqual, while XGBoost distributes importance a bit more evenly across additional features like overallcond, lotarea, and 1stflrsf.
- overallcond is ranked higher by XGBoost than Random Forest, suggesting it captures more subtle condition nuances XGBoost may handle better.

#### Final Takeaway
- These insights validate our earlier EDA conclusions — confirming that quality, size, and condition are core price drivers.
- SHAP strengthens our model explainability, offering transparency and interpretability critical for client trust and business deployment.

## Save Results

We will now retain the trained models, processed data, and pertinent outputs, like evaluation metrics and feature importance statistics, to guarantee reproducibility and make additional analysis easier. These saved files can be used in notebooks or projects in the future.

In [ ]:
import os
import pandas as pd
import joblib

# Define output directories
metrics_path = "outputs/metrics"
feature_importance_path = "outputs/ft_importance"
shap_values_path = "outputs/shap_values"
processed_data_path = "data/processed/final"
models_dir = "outputs/models"

# Create directories if they don't exist
os.makedirs(metrics_path, exist_ok=True)
os.makedirs(feature_importance_path, exist_ok=True)
os.makedirs(shap_values_path, exist_ok=True)
os.makedirs(processed_data_path, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

# --- Save Evaluation Metrics ---
results_df.to_csv(f"{metrics_path}/consolidated_model_performance.csv", index=False)
cv_results_df.to_csv(f"{metrics_path}/cross_validation_results.csv", index=False)
test_results_df.to_csv(f"{metrics_path}/test_set_results.csv", index=False)

# --- Save Feature Importances ---
importance_df.to_csv(f"{feature_importance_path}/random_forest_importance.csv", index=False)
gbr_importance_df.to_csv(f"{feature_importance_path}/xgboost_importance.csv", index=False)

# --- Save SHAP Summary Tables ---
shap_rf_table.to_csv(f"{shap_values_path}/shap_summary_random_forest.csv", index=False)
shap_importance_gbr_df.to_csv(f"{shap_values_path}/shap_summary_xgboost.csv", index=False)

# --- Save Final Datasets ---
X_train.to_csv(f"{processed_data_path}/X_train.csv", index=False)
X_test.to_csv(f"{processed_data_path}/X_test.csv", index=False)
pd.Series(y_train).to_csv(f"{processed_data_path}/y_train.csv", index=False)
pd.Series(y_test).to_csv(f"{processed_data_path}/y_test.csv", index=False)

# --- Save Tuned Models ---
joblib.dump(best_rf_model, f"{models_dir}/best_random_forest.pkl")
joblib.dump(best_dt_model, f"{models_dir}/best_decision_tree.pkl")
joblib.dump(best_gbr_model, f"{models_dir}/best_gradient_boosting.pkl")
joblib.dump(best_ridge_model, f"{models_dir}/best_ridge.pkl")
joblib.dump(best_svr_model, f"{models_dir}/best_svr.pkl")

try:
    # All your saving logic here

    print("All key outputs and trained models saved successfully.")

except Exception as e:
    print(f"[ERROR] Something went wrong: {e}")
    

## Next Steps: 

### Transition to Final Pipeline & Deployment

With the completion of model training, evaluation, interpretability analysis, and final output saving, we are now ready to proceed with the next stage of the project:

**1. Final Pipeline Construction**

- Build a unified pipeline integrating preprocessing and the best-performing model.
- Ensure it handles real-world inputs and edge cases gracefully.
- Validate with unseen/inherited data.

**2. Model and Artifact Serialization**

- Use joblib to save the entire pipeline object for reuse.
- Save supporting artefacts: scalers, encoders, column orders, and metadata.

**3. Deployment Preparation**

- Develop a prediction function that takes user input and returns a clean prediction with interpretations.
- Prepare for either:

    - Jupyter dashboard (streamlit/voila)
    - Flask/Django API setup (if desired)

**4. Documentation**

- Summarize business insights, technical implementation, and outcomes.
- Prepare README and technical appendix for Code Institute submission.